<a href="https://colab.research.google.com/github/vamsiakula1/Collage-rating-System/blob/main/imageclassify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipython-autotime
%load_ext autotime

In [ ]:
!pip install bing-image-downloader

In [ ]:
from bing_image_downloader import downloader

In [ ]:
downloader.download("pretty sunflower",limit=30,  output_dir='images', adult_filter_off=True, force_replace=False, timeout=60)

In [ ]:
downloader.download("icecream cone",limit=30,  output_dir='images', adult_filter_off=True, force_replace=False, timeout=60)

In [ ]:
downloader.download("rugby ball leather",limit=30,  output_dir='images', adult_filter_off=True, force_replace=False, timeout=60)

In [ ]:
#preprocessing
#resize
#flattening

import os
import matplotlib.pyplot as plot
import numpy as np
from skimage.io import imread
from skimage.transform import resize

In [ ]:
target=[]
images=[]
flat_data=[]

DATADIR='/content/images'

CATEGORIES=['pretty sunflower','icecream cone','rugby ball leather']
for category in CATEGORIES:
  class_num=CATEGORIES.index(category) #label encoding the values
  path=os.path.join(DATADIR,category)
  for img in os.listdir(path):
    img_array=imread(os.path.join(path,img))
    img_resized=resize(img_array,(150,150,3)) #Normalises to 0 t0 1
    flat_data.append(img_resized.flatten())
    images.append(img_resized)
    target.append(class_num)

flat_data=np.array(flat_data)
target=np.array(target)
images=np.array(images)


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(flat_data,target,test_size=0.3,random_state=109)

In [ ]:
#splitting the data training and testing
from sklearn.model_selection import GridSearchCV
from sklearn import svm
param_grid=[{'C':[1,10,100,1000],'kernel':['linear']},{'C':[1,10,100,1000],'gamma':[0.001,0.0001],'kernel':['rbf']}]

svc=svm.SVC(probability=True)
clf=GridSearchCV(svc,param_grid)
clf.fit(x_train,y_train)

In [ ]:
y_pred=clf.predict(x_test)
y_pred

In [ ]:
y_test

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [ ]:
accuracy_score(y_pred,y_test)

In [ ]:
confusion_matrix(y_pred,y_test)

In [ ]:
import pickle
pickle.dump(clf,open('img_model.p','wb'))

In [ ]:
model=pickle.load(open('img_model.p','rb'))

In [ ]:
#testing image
flat_data=[]
url=input("enter the url")
img=imread(url)
img_resized=resize(img,(150,150,3))
flat_data.append(img_resized.flatten())
flat_data=np.array(flat_data)
print(img.shape)
plot.imshow(img_resized)
y_out=model.predict(flat_data)
y_out=CATEGORIES[y_out[0]]
print(f'PREDICTED OUTPUT:{y_out}')

In [ ]:
!pip install streamlit

In [1]:
!pip install pyngrok==4.1.1
from pyngrok import ngrok

In [2]:
%%writefile app.py
import streamlit as st
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import pickle
from PIL import Image
st.title('image classifier')
st.text('upload the image')
model=pickle.load(open('img_model.p','rb'))
uploaded_file=st.file_uploader("Choose an image...",type="jpg")
if uploaded_file is not None:
  img=Image.open(uploaded_file)
  st.image(img,caption='Uploaded Image')

  if st.button('PREDICT'):
    CATEGORIES=['pretty sunflower','icecream cone','rugby ball leather']
    st.write('Result...')
    flat_data=[]
    img=np.array(img) 
    img_resized=resize(img,(150,150,3))
    flat_data.append(img_resized.flatten())
    flat_data=np.array(flat_data)
   
   
    y_out=model.predict(flat_data)
    y_out=CATEGORIES[y_out[0]]
    st.title(f'PREDICTED OUTPUT:{y_out}')
    q=model.predict_proba(flat_data)
    for index,item in enumerate(CATEGORIES):
      st.write(f'{item}:{q[0][index]*100}')




Overwriting app.py


In [4]:
!nohup streamlit run app.py &

url=ngrok.connect(port='8501')
url

nohup: appending output to 'nohup.out'


'http://3670b0e1ef03.ngrok.io'